# Customer Support

In [1]:
from crewai import Agent, Task, Crew

In [2]:
import os
from dotenv import load_dotenv

dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))

load_dotenv(dotenv_path)

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

In [3]:
support_agent = Agent(
    role="Senior Support Representative",  # use senior here to expect more polished responses
    goal="Be the most friendly and helpful "  # dont want model to be manipulated by user or spit out responses in tones we dont want it to
    "support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        " are now working on providing "
        "support to {customer}, a super important customer "
        " for your company."
        "You need to make sure that you provide the best support!"
        "Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    allow_delegation=False,  # cannot delegate tasks
    verbose=True,
)

In [4]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",  # double checks representative's responses are correct and high-quality
    goal="Get recognition for providing the "
    "best support quality assurance in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        "are now working with your team "
        "on a request from {customer} ensuring that "
        "the support representative is "
        "providing the best support possible.\n"
        "You need to make sure that the support representative "
        "is providing full"
        "complete answers, and make no assumptions."
    ),
    verbose=True,  # this agent CAN delegate tasks; can do its work and give it back to senior support representative if it needs to
)

In [5]:
# now to implement tools, finally!
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

/Users/marconardoneguerra/development/CrewAI/.venv/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [6]:
# possible tools can be built such as
# - Load customer data
# - Tap into previous conversations
# - Load data from a CRM
# - Check existing bug reports
# - Check existing feature requests
# - Check ongoing tickets
# - ... and more

In [7]:
search_tool = SerperDevTool()

In [8]:
scrape_tool = ScrapeWebsiteTool()

In [9]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/en/quickstart"
)

In [10]:
# You can give an agent tools either at the agent level or at the task level
# Giving it to the task restricts that tool usage to taht task
# Giving it to the agent allows the agent to use the tool whenever it wants

In [11]:
# Inquiry resolution task
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know "
        "to provide the best support possible."
        "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly "
        "tone throughout."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,  # this agent will use the doc_scrape_tool to find the appropriate documentation to help the user
)

In [12]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative.\n"
        "Ensure that the answer is comprehensive, accurate, and adheres "
        "to high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
        "thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        "find the information, "
        "ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
        "relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company "
        "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,  # this task purely assesses the support agent's response, does not have tools
)

In [13]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True,
    memory=True,  # creates chroma vectror db (short, long, and entity memories activated)
)

In [14]:
inputs = {
    "customer": "ethree solutions",
    "person": "Marco Nardone",
    "inquiry": "I need help with setting up a crew and kicking it off specifically, can you provide guidance in this subject?",
}

In [15]:
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: be0c9e83-c292-446e-85ff-bdd964460799                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Include a ready-to-import crewAI config (JSON/YAML) and copy-pasteable DSL so the recipient can deploy       │
│  immediately.                                                                                                   │
│  - Add concrete, runnable test scripts (curl/Postman/HTTPie) for every external integration (Zendesk, Jira,     │
│  Pinecone/Milvus) with expected responses and error-handling examples.                                          │
│  - Provide a small labeled eval dataset (50–100 queries) with expected outputs and a script to run automated    │
│  evals and compute metrics (accuracy, hallucination rate, latency).                                             │
│  - Add expl...                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 731.42ms ────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: ethree solutions just reached out with a super important ask:                                            │
│  I need help with setting up a crew and kicking it off specifically, can you provide guidance in this subject?  │
│                                                                                                                 │
│  Marco Nardone from ethree solutions is the one that reached out. Make sure to use everything you know to       │
│  provide the best support possible.You must strive to provide a complete and accurate response to the           │
│  customer's inquiry.                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

/Users/marconardoneguerra/development/CrewAI/.venv/lib/python3.13/site-packages/chromadb/types.py:144: 
PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you 
should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

/Users/marconardoneguerra/development/CrewAI/.venv/lib/python3.13/site-packages/chromadb/types.py:144: 
PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you 
should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 084bcb67-4ff8-4d84-854d-ab358206f333                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: be0c9e83-c292-446e-85ff-bdd964460799                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ---                                                                                              │
│                                                                                                                 │
│  Thank you for your patience, Marco! I’ve gathered all the necessary information to help you set up a crew in   │
│  crewAI effectively. Below, you'll find a comprehensive and detailed guide, which includes reference links,     │
│  example configurations, and best practices tailored to your needs.                                             │
│                                                                                                                 │
│  ### Complete Guidance for Setting Up a Crew in crewAI                                                          │
│                                                                                                                 │
│  **High-Level Overview of a Crew**                                                                              │
│  - A crew is essentially your AI-powered assistant in crewAI, composed of:                                      │
│    - **Chains**: Workflows made up of various actions (like retrieval, model calls, etc.).                      │
│    - **Policy**: Defines runtime behaviors, including fallback rules and retrieval settings.                    │
│    - **Config**: Includes models, tools, and credentials.                                                       │
│                                                                                                                 │
│  For in-depth details about these concepts, refer to the [Core                                                  │
│  Concepts](https://docs.crewai.com/en/introduction).                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Step-by-Step Checklist to Create and Kick Off a Crew                                                       │
│                                                                                                                 │
│  #### Step A: Create the Crew (Metadata + Defaults)                                                             │
│  1. **Metadata Configuration**                                                                                  │
│     - **Name**: "Support Triage Crew"                                                                           │
│     - **Description**: "Triage incoming support requests, answer FAQs, and create tickets for escalations."     │
│                                                                                                                 │
│  2. **Default Settings**                                                                                        │
│     - **Model**: Set to `gpt-4o` or your organization's preferred LLM.                                          │
│     - **Temperature**: Use a low temperature (0.0 to 0

In [16]:
from IPython.display import Markdown

Markdown(result.raw)

---

Thank you for your patience, Marco! I’ve gathered all the necessary information to help you set up a crew in crewAI effectively. Below, you'll find a comprehensive and detailed guide, which includes reference links, example configurations, and best practices tailored to your needs.

### Complete Guidance for Setting Up a Crew in crewAI

**High-Level Overview of a Crew**  
- A crew is essentially your AI-powered assistant in crewAI, composed of:
  - **Chains**: Workflows made up of various actions (like retrieval, model calls, etc.).
  - **Policy**: Defines runtime behaviors, including fallback rules and retrieval settings.
  - **Config**: Includes models, tools, and credentials.

For in-depth details about these concepts, refer to the [Core Concepts](https://docs.crewai.com/en/introduction).

---

### Step-by-Step Checklist to Create and Kick Off a Crew

#### Step A: Create the Crew (Metadata + Defaults)
1. **Metadata Configuration**
   - **Name**: "Support Triage Crew"
   - **Description**: "Triage incoming support requests, answer FAQs, and create tickets for escalations."
   
2. **Default Settings**
   - **Model**: Set to `gpt-4o` or your organization's preferred LLM.
   - **Temperature**: Use a low temperature (0.0 to 0.2) for factual tasks.
   - **Max Tokens**: Set between 512 and 1024, depending on expected output.

Please refer to the detailed instructions in the [Getting Started](https://docs.crewai.com/en/quickstart) section for a full overview.

#### Step B: Configure Tools and Credentials
1. **Add Necessary Tools:**
   - Integrate external systems (e.g., Zendesk, Jira) to enhance your crew’s functionalities.
   - Safeguard your API keys using environment variables.

2. **Test Connectivity:**  
   Use tools like curl or Postman to validate API keys.  
   **Example CURL command for testing Zendesk:**
   ```bash
   curl -s -u 'your_email/token:YOUR_ZENDESK_API_TOKEN' \
     -X GET "https://your_subdomain.zendesk.com/api/v2/tickets.json"
   ```

Refer to [Add Tools & Credentials](https://docs.crewai.com/en/quickstart#add-tools-credentials) for detailed procedures.

#### Step C: Build the Initial Chain(s)
1. **Minimal Chain:** Start with a chain that answers FAQs:
   - **Retriever:** Fetch the top_k (e.g., 3) relevant documents.
   - **LLM:** Use a prompt that relies solely on retrieved passages.
   - **Output:** Return an assistant's reply based directly on fetched data.

**Example Chain Setup:**
```yaml
chains:
  - name: support_simple_reply
    steps:
      - id: fetch_passages
        type: retriever
        config: { top_k: 3 }
      - id: generate_response
        type: llm
        prompt: |
          System: "You are a support assistant. Use only the retrieved passages as facts. If the answer is not found, say 'I don't know.'"
          User: "{user_message}\n\nRetrieved passages:\n{fetched_passages}"
      - id: output_response
        type: output
```

#### Step D: Configure Policy
- You can choose a **simple policy** for straightforward tasks or a **Plan-Then-Act** approach for complex workflows.
- Set parameters (top_k, chunk_size, overlap) to enhance response quality, and establish thresholds for model confidence to determine fallback responses when necessary.

Refer to [Configure Policy](https://docs.crewai.com/en/quickstart#configure-policy) for more details.

#### Step E: Define Tool Actions
- Create specific actions for various tools (such as ticket creation).
- Ensure that outputs from these actions are logged, and monitor for errors or failures.

**Example Tool Action for Ticket Creation:**
```yaml
tools:
  zendesk_create_ticket:
    type: tool
    action: create_ticket
    input:
      subject: "{user_message}"
      description: "{context}"
  output_mapping:
    ticket_id: response.ticket.id
```

#### Step F: Testing and Validation
1. **Init Tests:** Test each tool action individually to ensure proper functionality.
2. **Sample Conversations:** Test with 50-100 labeled queries for a smooth flow.
3. **Telemetry Monitoring:** Implement interaction logs for debugging purposes.

Refer to [Test the Crew](https://docs.crewai.com/en/quickstart#test-the-crew) for testing guidelines.

#### Step G: Deployment and Monitoring
- Rollout phases: **Internal alpha → Limited beta → Gradual customer rollout.**
- It's crucial to implement monitoring for performance, user feedback, and continuous improvement.

**Suggested Monitoring Metrics:**
- Accuracy rates from sample evaluations.
- Response time (latency).
- User satisfaction (CSAT).

### Example Crew Configuration in YAML
Here’s a quick condensed version ready for implementation:
```yaml
crew:
  name: "Support Triage Crew"
  model: "gpt-4o"
  temperature: 0.2
  tools:
    zendesk:
      type: tool
      api_key: "YOUR_API_KEY"
  chains:
    - name: support_chain
      steps:
        - id: fetch_passages
          action: retriever
        - id: process_request
          action: llm
        - id: output
          action: output
```

### Next Steps for Your Crew
Marco, to better tailor this configuration, please respond to the following:
1. Which systems will you integrate first (e.g., Zendesk, Jira)?
2. What specific user queries should the crew be prepared to handle?
3. Should the crew create tickets automatically for low-confidence responses or ask for confirmation?
4. What is the sensitivity level of your data regarding PII (Personally Identifiable Information)?

Once I receive your feedback, I can provide further detailed next steps, tailored prompts, full crew configuration, and testing guidelines to assist you in your setup.

### References:
- [crewAI Quickstart](https://docs.crewai.com/en/quickstart) - The full starting guide for crewAI.
- [Core Concepts](https://docs.crewai.com/en/introduction) - Definitions and foundational elements.

I look forward to your responses so we can customize this setup and get you rolling with crewAI! Thank you for choosing us for your support solutions.